In [ ]:
pip install soundstream

In [1]:
import torchaudio

from soundstream import from_pretrained, load

/home/girish/anaconda3/envs/soundstream/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
waveform = load('/home/girish/Girish/RESEARCH/depression_DATA_wavCODEC/edaic_segmented_5second/edaic_segmented_5second/segmented_files/test/620_AUDIO_62.wav')
audio_codec = from_pretrained() 

quantized = audio_codec(waveform, mode='encode')
recovered = audio_codec(quantized, mode='decode')

torchaudio.save('out.wav', recovered[0], 16000)

/home/girish/anaconda3/envs/soundstream/lib/python3.10/site-packages/soundstream/__init__.py:35: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(checkpoint_path, ma

In [10]:
print(quantized.squeeze().shape)

torch.Size([400, 256])


In [19]:
print(quantized.squeeze())

tensor([[-0.0225,  0.0979,  0.2217,  ...,  0.4354,  0.0518, -0.3353],
        [-0.0220, -0.0156,  0.0504,  ...,  0.2097, -0.5930,  0.0954],
        [ 0.1251, -0.0427,  0.0427,  ...,  0.1024, -1.0428,  0.0357],
        ...,
        [-0.1343,  0.3019, -0.0302,  ..., -0.7759, -0.1442,  0.3840],
        [ 0.2785, -0.6767, -0.2164,  ...,  1.1040, -0.8486,  0.3310],
        [ 0.1539, -0.1283, -0.1355,  ..., -0.3518,  0.6766,  0.2185]],
       grad_fn=<SqueezeBackward0>)


In [12]:
print(quantized.squeeze().float().mean(dim=0).shape)

torch.Size([256])


In [13]:
print(quantized.squeeze().float().mean(dim=0))

tensor([ 5.2620e-03, -1.3529e-02, -2.8629e-02, -1.0998e-02,  5.3564e-03,
         9.3098e-03, -4.0186e-02,  3.3174e-02,  2.5929e-02, -8.5583e-03,
         1.6324e-02, -9.7207e-03,  3.6596e-03, -3.2731e-02,  1.8637e-02,
         3.5543e-02, -2.5786e-02,  1.0750e-02, -6.8302e-04,  4.9442e-02,
         1.3519e-02, -2.1147e-02,  2.4792e-02,  2.1093e-02, -4.7846e-02,
        -2.4653e-02,  4.3333e-03, -5.9818e-03,  3.4861e-04,  1.1301e-02,
        -1.7666e-02, -1.0029e-02, -6.8289e-03, -2.1606e-02,  1.8513e-02,
        -1.1290e-02,  1.9513e-02, -2.9469e-02,  1.5081e-02, -2.1789e-02,
         6.9515e-03, -1.6408e-02, -8.3200e-03, -3.1011e-02, -1.9562e-02,
        -3.7069e-03, -3.2628e-02,  1.7460e-02, -1.4486e-02, -2.5599e-03,
         1.4377e-03, -2.5774e-02,  3.8303e-02,  1.5526e-02, -2.1759e-02,
        -1.7606e-02, -3.0932e-02, -2.0930e-02,  7.5524e-03,  1.4036e-03,
        -2.1100e-02, -1.0842e-02, -1.7531e-02,  9.6162e-03,  1.2854e-02,
        -2.0388e-02,  2.8979e-02,  4.1797e-03, -1.7

In [ ]:

waveform = load('/home/girish/Girish/RESEARCH/depression_DATA_wavCODEC/edaic_segmented_5second/edaic_segmented_5second/segmented_files/test/620_AUDIO_62.wav')
audio_codec = from_pretrained() 

quantized = audio_codec(waveform, mode='encode')
print(quantized.squeeze().float().mean(dim=0).shape)
print(quantized.squeeze().float().mean(dim=0))

In [ ]:
import os
import pandas as pd
import numpy as np
import torch
import torchaudio
from soundstream import from_pretrained, load

folder_path = r"wav folder path"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

filenames = []
quantized_features_list = []
quantized_avg_features_list = []

audio_codec = from_pretrained().to(device)

for filename in os.listdir(folder_path):
    if filename.endswith(".wav"):
        file_path = os.path.join(folder_path, filename)
        
        waveform = load(file_path).to(device)
        
        quantized = audio_codec(waveform, mode='encode')
        
        quantized_avg = quantized.squeeze().float().mean(dim=0).detach().cpu().numpy()
        
        filenames.append(filename)
        quantized_features_list.append(quantized.detach().cpu().numpy())  
        quantized_avg_features_list.append(quantized_avg)  

quantized_avg_features_df = pd.DataFrame(quantized_avg_features_list, columns=[f'{i}' for i in range(1, quantized_avg.shape[0] + 1)])
quantized_avg_features_df.insert(0, 'filename', filenames)

quantized_avg_features_df.to_csv(r'output file path.csv', index=False)

print("Averaged quantized features for all files have been saved to soundstream_heart.csv")
